In [1]:
import sys
sys.path.append('../src_variable')

import loading_json as load
import road_network as rn
import torch
import matplotlib.pyplot as plt

In [2]:
T, roads, junctions, ControlPoints = load.read_json('../configs_variable/simple_1-1.json')

1 roads initialized
2 roads initialized
All roads initialized
1 junctions initialized
All junctions initialized


In [3]:
def initialize_road_network(T, roads, junctions, vmaxes, control_points):
    '''
    Initializing a road network given a time, roads and junctions as well as the speed limits
    vmaxes.
    Could probably instead just update densities and speed limits of existing system...

    Need to be updated to take in vmaxes that may vary for each road
    '''

    n_speeds = len(control_points) + 1 # Number of different speed limits for each road

    loaded_roads = []
    loaded_junctions = []
    for l, r in enumerate(roads):
        # print(Vmax[l])
        idx = l * n_speeds
        loaded_roads.append(rn.Road(b=r['b'], L = r['L'], N = r['N'], Vmax=[float(vmaxes[i]) for i in range(idx, idx+n_speeds)],
                    scheme=r['Scheme'], limiter="minmod",
                    initial = r["Init_distr"], inflow = r["Inflow"]))

    for j in junctions:
        loaded_junctions.append(rn.Junction(roads = [loaded_roads[i] for i in j["j_roads"]],
                                        entering = j["entering"], leaving = j["leaving"],
                                        distribution=j["distribution"], redlights=[]))
    network = rn.RoadNetwork(loaded_roads, loaded_junctions, T, control_points)

    return loaded_roads, loaded_junctions, network

In [4]:
second = torch.tensor([80.0000, 80., 80., 50.0000, 40.0000, 35.0000])
_, _, network = initialize_road_network(T, roads, junctions, second, ControlPoints)

In [5]:
history = network.solve_cons_law()

Currently simulating untill time 10
Time equal to 0
Currently simulating untill time 40
Time equal to 10.0
Currently simulating untill time 60
Time equal to 40.0


In [6]:
network.roads[0].queue_length

tensor(0.1201, grad_fn=<AddBackward0>)

In [10]:
h = history[0]

for t in h.keys():
    if t == 10.0:
        print("BOOM")
    if t == 40.0:
        print("BO")
    if t == 60.0:
        print("bbb")

BOOM
BO
bbb


In [37]:
cycle = [1.,1.3,0.3, 1.,1.4, 0.1]
assert(len(cycle)%2 == 0)
cycle_time = sum(cycle)
print(cycle_time)
cycle = [torch.tensor(c, requires_grad=True) for c in cycle]

5.1


In [41]:
t_list = [0.1, 0.4, 1.3, 1.7, 2.2, 2.5, 2.6, 3.8]
for t in t_list:
    print(f"Current t: {t}")

    tau = t % cycle_time
    n = t // cycle_time

    # if True:#tau < cycle[0]:
    #     print(f"{(cycle_time*n, cycle_time*n + cycle[0].detach())}")
    #     print("Same state as start")

    # if True:#tau > cycle_time - cycle[-1]:
    #     print(f"{(cycle_time*n + cycle_time - cycle[-1].detach(), cycle_time*(n+1))}")
    #     print("Opposite of start")

    found = False
    combined = 0
    for i in range(0, len(cycle), 2):
        combined += cycle[i]
        if tau >= combined and tau < combined + cycle[i+1]:
            found = True
            print(f"{(cycle_time*n + combined, cycle_time*n + combined + cycle[i+1].detach())}")
            print("Opposite of start")

        combined += cycle[i+1]

    if not found:
        print("t in other intervals")


Current t: 0.1
t in other intervals
Current t: 0.4
t in other intervals
Current t: 1.3
(tensor(1., grad_fn=<AddBackward0>), tensor(2.3000, grad_fn=<AddBackward0>))
Opposite of start
Current t: 1.7
(tensor(1., grad_fn=<AddBackward0>), tensor(2.3000, grad_fn=<AddBackward0>))
Opposite of start
Current t: 2.2
(tensor(1., grad_fn=<AddBackward0>), tensor(2.3000, grad_fn=<AddBackward0>))
Opposite of start
Current t: 2.5
t in other intervals
Current t: 2.6
(tensor(2.6000, grad_fn=<AddBackward0>), tensor(3.6000, grad_fn=<AddBackward0>))
Opposite of start
Current t: 3.8
t in other intervals
